In [1]:
import logging
import urllib
from pathlib import Path

import requests
from config import Config
from requests_ntlm import HttpNtlmAuth

In [4]:
class OperatorData:
    def __init__(self):
        self.ssrs_url = Config.SSRS_BASE_URL + "MPI Reports/Master Operator Data"
        self.ssrs_usr = Config.SSRS_USERNAME
        self.ssrs_pwd = Config.SSRS_PASSWORD
        self.payload = [
            ("rs:Command", "Render"),
            ("rs:Format", "CSV"),
            ("rc:ItemPath", "table1"),
        ]

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )
        
        if response.status_code == 200:
            data = response.content.decode("utf8")
            return data
        return None

In [5]:
operators = OperatorData().get()

http://95.211.42.206/ReportServer_SSRS?/MPI%20Reports/Master%20Operator%20Data&rs%3ACommand=Render&rs%3AFormat=CSV&rc%3AItemPath=table1


In [6]:
import pandas as pd
import io

operator_df = pd.read_csv(io.StringIO(operators))

In [7]:
operator_df.head()

,OperatorID,ShortName,OperatorName,Currency,OperatorCode,Nationality,FirmName
0,1,MPCLM,"Meeting Point City, Lakes and Mountains GmbH",EUR,MP_CLM,DE,Meeting Point City Lakes and Mnts GmbH
1,2,MPUAE,Meeting Point Emirates,AED,MP_UA,AE,Meeting Point Emirates
2,3,MPLE,Meeting Point Lebanon SARL,NOK,MP_LE,NaN,Meeting Point Lebanon SARL
3,4,MPA,MEETING POINT ASIA,THB,MP_AS,TH,Meeting Point Asia
4,5,MPM,Meeting Point Malta Ltd.,EUR,MP_MT,MT,Meeting Point Malta Ltd


In [8]:
operator_df.to_csv("operators.csv", index=False)